<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_alternate_reality_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_alternate_reality_ai_fixed.py

Physics‐informed AI pipeline for AlternateRealityAI with corrected
plot_uncertainty (no NumPy–Tensor type mix).

1. Synthetic dataset of 5 universe parameters → 3 stability metrics
2. Physics‐informed residuals enforcing toy universe stability laws
3. MLP with LayerNorm & Dropout for robustness
4. MC‐Dropout inference for uncertainty quantification
5. Training loop with AdamW, ReduceLROnPlateau, gradient clipping, early stopping
6. Visualizations: loss curves, scatter plots, uncertainty heatmap
"""

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. Synthetic Alternate Reality Dataset
# ------------------------------------------------------------------------------
class AlternateRealityDataset(Dataset):
    def __init__(self, n_samples=6000, seed=0):
        np.random.seed(seed)
        # Inputs: c_scale, G_scale, QF_amp, curvature, entropy_density
        c    = np.random.uniform(0.5, 1.5,   (n_samples,1)).astype(np.float32)
        G    = np.random.uniform(0.1, 10.0,  (n_samples,1)).astype(np.float32)
        QF   = np.random.uniform(0.001,0.1,  (n_samples,1)).astype(np.float32)
        curv = np.random.uniform(-1.0,1.0,   (n_samples,1)).astype(np.float32)
        ent  = np.random.uniform(1e-5,1e-3,  (n_samples,1)).astype(np.float32)

        X_raw = np.hstack([c, G, QF, curv, ent])

        # Targets (toy physics laws)
        eps = 1e-6
        es = c / (G + eps)                         # expansion stability
        ci = 1.0 / (1.0 + QF * np.abs(curv))       # causality index
        cd = ent / (QF + eps)                      # coherence duration

        Y_raw = np.hstack([es, ci, cd]).astype(np.float32)
        Y_raw += 0.02 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape).astype(np.float32)

        # Compute normalization stats
        self.X_mean, self.X_std = X_raw.mean(0), X_raw.std(0) + 1e-6
        self.Y_mean, self.Y_std = Y_raw.mean(0), Y_raw.std(0) + 1e-6

        # Standardize
        self.X = (X_raw - self.X_mean) / self.X_std
        self.Y = (Y_raw - self.Y_mean) / self.Y_std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.X[idx]),
            torch.from_numpy(self.Y[idx])
        )

# ------------------------------------------------------------------------------
# 2. Model Definition
# ------------------------------------------------------------------------------
class AlternateRealityAI(nn.Module):
    def __init__(self, input_dim=5, hidden_dims=(64,64), output_dim=3, p_drop=0.1):
        super().__init__()
        layers, d = [], input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(d, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            d = h
        layers.append(nn.Linear(d, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics‐Informed Residual
# ------------------------------------------------------------------------------
def physics_residual(pred, X, stats):
    X_den = X * stats['X_std'] + stats['X_mean']
    c, G, QF, curv, ent = X_den.t()
    Y_den = pred * stats['Y_std'] + stats['Y_mean']
    es_pred, ci_pred, cd_pred = Y_den.t()

    eps = 1e-6
    es_true = c / (G + eps)
    ci_true = 1.0 / (1.0 + QF * curv.abs())
    cd_true = ent / (QF + eps)

    r1 = nn.MSELoss()(es_pred, es_true)
    r2 = nn.MSELoss()(ci_pred, ci_true)
    r3 = nn.MSELoss()(cd_pred, cd_true)
    return r1 + r2 + r3

def total_loss(pred, true, X, stats, lam=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, X, stats)
    return mse + lam * phys, mse, phys

# ------------------------------------------------------------------------------
# 4. MC‐Dropout Inference
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, x, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(x))
    arr = torch.stack(preds, 0)
    return arr.mean(0), arr.std(0)

# ------------------------------------------------------------------------------
# 5. Training Loop
# ------------------------------------------------------------------------------
def train(model, train_loader, val_loader, stats, device,
          lr=1e-3, wd=1e-5, lam=1.0, epochs=100, patience=10):
    model.to(device)
    opt   = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', factor=0.5, patience=5)

    best_val, wait = float('inf'), 0
    history = {'train':[], 'val':[]}

    for ep in range(1, epochs+1):
        model.train()
        t_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss, _, _ = total_loss(pred, yb, xb, stats, lam)
            opt.zero_grad(); loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            t_loss += loss.item() * xb.size(0)
        t_loss /= len(train_loader.dataset)

        model.eval()
        v_loss = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss, _, _ = total_loss(pred, yb, xb, stats, lam)
                v_loss += loss.item() * xb.size(0)
        v_loss /= len(val_loader.dataset)

        sched.step(v_loss)
        history['train'].append(t_loss)
        history['val'].append(v_loss)
        print(f"Epoch {ep:03d} | Train {t_loss:.4e} | Val {v_loss:.4e}")

        if v_loss + 1e-8 < best_val:
            best_val, wait = v_loss, 0
            torch.save(model.state_dict(), "best_alt_reality.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {ep}")
                break

    model.load_state_dict(torch.load("best_alt_reality.pth"))
    return history

# ------------------------------------------------------------------------------
# 6. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_history(hist):
    plt.figure()
    plt.plot(hist['train'], label='Train')
    plt.plot(hist['val'],   label='Val')
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.show()

def plot_scatter(y_true, y_pred, name):
    plt.figure()
    plt.scatter(y_true, y_pred, s=5, alpha=0.6)
    m, M = y_true.min(), y_true.max()
    plt.plot([m, M], [m, M], 'r--')
    plt.title(name); plt.xlabel("True"); plt.ylabel("Pred"); plt.show()

def plot_uncertainty(model, stats, device):
    # Generate grid in NumPy for plotting axes
    cs  = np.linspace(0.5, 1.5, 100, dtype=np.float32)
    Gs  = np.linspace(0.1, 10.0, 100, dtype=np.float32)
    Cg, Gg = np.meshgrid(cs, Gs)

    # Build a torch tensor grid on the correct device
    pts = Cg.size
    grid_t = torch.zeros((pts, 5), device=device, dtype=torch.float32)
    grid_t[:, 0] = torch.from_numpy(Cg.ravel()).to(device)
    grid_t[:, 1] = torch.from_numpy(Gg.ravel()).to(device)
    for i in (2, 3, 4):
        grid_t[:, i] = stats['X_mean'][i]

    # Normalize in torch
    Xn = (grid_t - stats['X_mean']) / stats['X_std']

    # MC-Dropout inference
    _, std = mc_dropout_predict(model, Xn, T=100)
    std_map = std[:, 0].cpu().numpy().reshape(Cg.shape)

    # Plot heatmap
    plt.figure(figsize=(6,5))
    plt.pcolormesh(Cg, Gg, std_map, cmap='inferno', shading='auto')
    plt.colorbar(label='Std Expansion Stability')
    plt.xlabel('c_scale'); plt.ylabel('G_scale')
    plt.title('Uncertainty Heatmap: Expansion Stability')
    plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ds = AlternateRealityDataset(6000)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, device=device),
        'X_std' : torch.tensor(ds.X_std,  device=device),
        'Y_mean': torch.tensor(ds.Y_mean, device=device),
        'Y_std' : torch.tensor(ds.Y_std,  device=device),
    }

    n_val = int(0.2 * len(ds))
    tr, va = random_split(ds, [len(ds) - n_val, n_val])
    tr_ld = DataLoader(tr, batch_size=128, shuffle=True)
    va_ld = DataLoader(va, batch_size=256)

    model = AlternateRealityAI().to(device)
    history = train(model, tr_ld, va_ld, stats, device)

    plot_history(history)

    # Scatter plots
    X_all = torch.from_numpy(ds.X).float().to(device)
    with torch.no_grad():
        Yp_norm = model(X_all).cpu().numpy()
    Yp = Yp_norm * ds.Y_std + ds.Y_mean
    Yt = ds.Y
    for i, name in enumerate(['Expansion Stability','Causality Index','Coherence Duration']):
        plot_scatter(Yt[:,i], Yp[:,i], name)

    # Uncertainty heatmap
    plot_uncertainty(model, stats, device)